In [1]:
#we will classify MNIST
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, parser='auto', return_X_y=True)
X = X.values
y = y.astype(int).values

#check the dimensions of our data
print(X.shape, y.shape)

(70000, 784) (70000,)


In [2]:
#standardize our features to stabilize gradient descent, pixel values in [-1, 1]
X = 2*((X/255.)-0.5)

#perform train-test split: 10000 test set, 5000 validation set
from sklearn.model_selection import train_test_split

X_inter, X_test, y_inter, y_test = train_test_split(X, y,\
                                                    test_size=10000, random_state=42, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_inter, y_inter,\
                                                      test_size=5000, random_state=42, stratify=y_inter)


In [3]:
import numpy as np
from neuralnet import NeuralNetMLP

######################
#some useful functions
######################

#computes sigmoid function
def sigmoid(z):
    return 1./(1.+np.exp(-z))

#converts integer class labels to one-hot encoding
def int_to_onehot(y, numlabels):
    ohe = np.zeros((y.shape[0], num_labels))
    for i, val in enumerate(y):
        ohe[i, val] = 1
        
    return ohe

#returns the mean squared error loss
def mse_loss(goals, probs, num_labels=10):
    targets=int_to_onehot(goals, num_labels)
    return np.mean((targets-probs)**2)

#returns accuracy
def accuracy(targets, pred_labels):
    return np.mean(pred_labels == targets)

In [ ]:
#instantiate an MLP model
mlp = NeuralNetMLP(num_features=28**2, num_hidden=50, num_classes=10)

#function to implement mini-batch gradient descent
def minibatch_generator(X, y, minibatch_size):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    for index in range(0, indices.shape[0]-minibatch_size+1, minibatch_size):
        batch_index = indices[start_index:start_index+minibatch_size]
        yield X[batch_index], y[batch_index]

#function to calculate evaluation metrics
def compute_mse_and_acc(nnet, X, y, num_labels=10, minibatch_size=100):
    